In [1]:
#Import all packages

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

test = pd.read_csv('final_data/test_nolabel.csv')
train = pd.read_csv('final_data/train_outcome.csv')
pat_1 = pd.read_csv('final_data/x_all/1.txt', delimiter = ",")
pat_1

,Hour,Variable,Value
0,1,Age,50.73
1,1,Gender,1.00
2,1,ID,1.00
3,2,Age,50.73
4,2,DBP,51.31
...,...,...,...
131,17,HR,73.84
132,17,MAP,76.73
133,17,O2Sat,96.09
134,17,Resp,14.87


In [2]:
test.head()

,ID,Outcome,Score
0,1,NaN,NaN
1,8,NaN,NaN
2,9,NaN,NaN
3,10,NaN,NaN
4,11,NaN,NaN


In [3]:
train.describe()

,ID,Outcome
count,15144.000000,15144.000000
mean,10822.677298,0.136358
std,6226.623458,0.343179
min,2.000000,0.000000
25%,5459.750000,0.000000
50%,10812.500000,0.000000
75%,16201.250000,0.000000
max,21633.000000,1.000000


15,144 patients in the training set. 2065 have septis, 13078 do not.

* If have septis, outcome equal 1

We first will play aorund with one of the txt files in order to manipulate the data into 1 row, using the median of each variable. We also want to add Hour, which is a column in the txt file, as an explanatory variable

In [4]:
pat_1

,Hour,Variable,Value
0,1,Age,50.73
1,1,Gender,1.00
2,1,ID,1.00
3,2,Age,50.73
4,2,DBP,51.31
...,...,...,...
131,17,HR,73.84
132,17,MAP,76.73
133,17,O2Sat,96.09
134,17,Resp,14.87


* Each txt had different measures of different variables over a given timeframe. For patient 1, we see that there are 17 hours.

* 136 rows (which will vary for each) by 3 columns: The three columns are: Hour, Variable, Value

In [5]:
p = pat_1.groupby(['Variable']).median().transpose()
p['Hour'] = (pat_1['Hour']).median()
p = p.drop(['Hour'], axis = 0)
p.head()

Variable,Age,BUN,Creatinine,DBP,Gender,HR,Hct,Hgb,ID,MAP,Magnesium,O2Sat,Phosphate,Platelets,Potassium,Resp,SBP,Temp,WBC,Hour
Value,50.73,15.0,0.5,60.685,1.0,66.12,40.9,13.2,1.0,77.08,1.8,97.95,2.9,225.0,4.2,17.46,99.2,36.47,13.2,9.0


Aside from getting the median, we also are going to get the min and max of each variable measure for the patient

In [6]:
maxes = pat_1.groupby('Variable').max()
mins = pat_1.groupby('Variable').min()

In [7]:
maxes = maxes.drop(['Hour'], axis = 1).transpose()
maxes['Hour'] = max(pat_1['Hour'])
maxes = maxes.add_suffix('_max')
maxes

Variable,Age_max,BUN_max,Creatinine_max,DBP_max,Gender_max,HR_max,Hct_max,Hgb_max,ID_max,MAP_max,Magnesium_max,O2Sat_max,Phosphate_max,Platelets_max,Potassium_max,Resp_max,SBP_max,Temp_max,WBC_max,Hour_max
Value,50.73,15.0,0.5,78.62,1.0,81.82,40.9,13.2,1.0,92.72,1.8,100.03,2.9,225.0,4.2,24.02,138.79,36.59,13.2,17


In [8]:
mins = mins.drop(['Hour'], axis = 1).transpose()
mins['Hour'] = min(pat_1['Hour'])
mins = mins.add_suffix('_min')
mins

Variable,Age_min,BUN_min,Creatinine_min,DBP_min,Gender_min,HR_min,Hct_min,Hgb_min,ID_min,MAP_min,Magnesium_min,O2Sat_min,Phosphate_min,Platelets_min,Potassium_min,Resp_min,SBP_min,Temp_min,WBC_min,Hour_min
Value,50.73,15.0,0.5,47.22,1.0,62.81,40.9,13.2,1.0,58.35,1.8,96.09,2.9,225.0,4.2,14.87,78.92,35.97,13.2,1


In [9]:
min_max = pd.concat([mins, maxes], axis=1)
min_max.head()

Variable,Age_min,BUN_min,Creatinine_min,DBP_min,Gender_min,HR_min,Hct_min,Hgb_min,ID_min,MAP_min,...,Magnesium_max,O2Sat_max,Phosphate_max,Platelets_max,Potassium_max,Resp_max,SBP_max,Temp_max,WBC_max,Hour_max
Value,50.73,15.0,0.5,47.22,1.0,62.81,40.9,13.2,1.0,58.35,...,1.8,100.03,2.9,225.0,4.2,24.02,138.79,36.59,13.2,17


* Now, we will combine all 3 (median, max, min) together

In [10]:
p = pd.concat([p, min_max], axis=1)
p.shape

(1, 60)

In [11]:
p.columns

Index(['Age', 'BUN', 'Creatinine', 'DBP', 'Gender', 'HR', 'Hct', 'Hgb', 'ID',
       'MAP', 'Magnesium', 'O2Sat', 'Phosphate', 'Platelets', 'Potassium',
       'Resp', 'SBP', 'Temp', 'WBC', 'Hour', 'Age_min', 'BUN_min',
       'Creatinine_min', 'DBP_min', 'Gender_min', 'HR_min', 'Hct_min',
       'Hgb_min', 'ID_min', 'MAP_min', 'Magnesium_min', 'O2Sat_min',
       'Phosphate_min', 'Platelets_min', 'Potassium_min', 'Resp_min',
       'SBP_min', 'Temp_min', 'WBC_min', 'Hour_min', 'Age_max', 'BUN_max',
       'Creatinine_max', 'DBP_max', 'Gender_max', 'HR_max', 'Hct_max',
       'Hgb_max', 'ID_max', 'MAP_max', 'Magnesium_max', 'O2Sat_max',
       'Phosphate_max', 'Platelets_max', 'Potassium_max', 'Resp_max',
       'SBP_max', 'Temp_max', 'WBC_max', 'Hour_max'],
      dtype='object', name='Variable')

In [12]:
p.head()

Variable,Age,BUN,Creatinine,DBP,Gender,HR,Hct,Hgb,ID,MAP,...,Magnesium_max,O2Sat_max,Phosphate_max,Platelets_max,Potassium_max,Resp_max,SBP_max,Temp_max,WBC_max,Hour_max
Value,50.73,15.0,0.5,60.685,1.0,66.12,40.9,13.2,1.0,77.08,...,1.8,100.03,2.9,225.0,4.2,24.02,138.79,36.59,13.2,17


### Accounting for the Longitudinal Data: 

* To account for lag, I divided the difference in the last measurement and the first measurement by the number of measurements that were taken

In [13]:
#To capture Longitudinal Information, instead of using rolling window, we will do first and last

#Already have min and max hour, so dont have to add here

first = pat_1.groupby('Variable').first().transpose()
first.drop(['Hour'], axis = 0, inplace=True)

In [14]:
last = pat_1.groupby('Variable').last().transpose()
last.drop(['Hour'], axis = 0, inplace = True)

In [15]:
diff = last-first
diff

Variable,Age,BUN,Creatinine,DBP,Gender,HR,Hct,Hgb,ID,MAP,Magnesium,O2Sat,Phosphate,Platelets,Potassium,Resp,SBP,Temp,WBC
Value,0.0,0.0,0.0,14.08,0.0,2.36,0.0,0.0,0.0,11.36,0.0,-2.81,0.0,0.0,0.0,-0.94,6.54,-0.5,0.0


In [16]:
count = pat_1.groupby('Variable').count().transpose()
count.drop(['Hour'], axis = 0, inplace = True)

In [17]:
d = diff.loc['Value'] / count.loc['Value']
avg_diff = pd.DataFrame(d).transpose().add_suffix('_ad')
avg_diff

Variable,Age_ad,BUN_ad,Creatinine_ad,DBP_ad,Gender_ad,HR_ad,Hct_ad,Hgb_ad,ID_ad,MAP_ad,Magnesium_ad,O2Sat_ad,Phosphate_ad,Platelets_ad,Potassium_ad,Resp_ad,SBP_ad,Temp_ad,WBC_ad
Value,0.0,0.0,0.0,1.005714,0.0,0.1475,0.0,0.0,0.0,0.757333,0.0,-0.187333,0.0,0.0,0.0,-0.067143,0.436,-0.166667,0.0


In [18]:
p = pd.concat([p, avg_diff], axis=1)

In [19]:
p.set_index('ID')

Variable,Age,BUN,Creatinine,DBP,Gender,HR,Hct,Hgb,MAP,Magnesium,...,MAP_ad,Magnesium_ad,O2Sat_ad,Phosphate_ad,Platelets_ad,Potassium_ad,Resp_ad,SBP_ad,Temp_ad,WBC_ad
ID,,,,,,,,,,,,,,,,,,,,,
1.0,50.73,15.0,0.5,60.685,1.0,66.12,40.9,13.2,77.08,1.8,...,0.757333,0.0,-0.187333,0.0,0.0,0.0,-0.067143,0.436,-0.166667,0.0


* Now that we see how this will work for one patient file, we need to write a loop that does this for all of our txt files. Each row in the final dataframe will represent a patient with the given ID, and will have the median, max, and min measures of the variables recorded, along with the min, max, and median time.

In [20]:
#Write a loop to use averages for every file
import glob

#Defining the path of the file
path = 'final_data/x_all'
#Entire Folder
file_list = glob.glob(path + "/*.txt")



#Blank DF that we will concat to
main = pd.DataFrame()

#Writing the loop
for i in range(0,len(file_list)): 
#Iterate over entirity of list (len(file_list) is final)
    
    #Load in individual patient file
    df_org = pd.read_csv(file_list[i], delimiter = ",")
    
    #Get our three DF based on the summary statistics and transpose them
    df = df_org.groupby(['Variable']).median().transpose()
    maxes = df_org.groupby('Variable').max().transpose()
    mins = df_org.groupby('Variable').min().transpose()
    first = df_org.groupby('Variable').first().transpose()
    last = df_org.groupby('Variable').last().transpose()
    count = df_org.groupby('Variable').count().transpose()

    #Add Hour Column to transposed median matrix (add column suffixes for simplicity)
    df['Hour'] = df_org['Hour'].median()
    maxes['Hour'] = max(pat_1['Hour'])
    maxes = maxes.add_suffix('_max')
    mins['Hour'] = min(pat_1['Hour'])
    mins = mins.add_suffix('_min')
    
    
    #Drop Hour row to get each patient into one row (since it was transposed, must drop this other row)
    df = df.drop(['Hour'], axis = 0)
    maxes = maxes.drop(['Hour'], axis = 0)
    mins = mins.drop(['Hour'], axis = 0)
    first = first.drop(['Hour'], axis = 0)
    last = last.drop(['Hour'], axis = 0)
    count = count.drop(['Hour'], axis = 0)
    
    #Get Avg
    diff = last-first
    
    #Getting the avg diff DF
    d = diff.loc['Value'] / count.loc['Value']
    avg_diff = pd.DataFrame(d).transpose().add_suffix('_ad')
    
    #Merging All 3 Together side by side so we still have one row
    min_max = pd.concat([mins, maxes], axis=1)
    min_max_avg = pd.concat([min_max, avg_diff], axis=1)
    df = pd.concat([df, min_max_avg], axis=1)
    
    #Add each patient to the previous
    main = pd.concat([main, df], axis = 0)

    
#Total Run Time = About 20 minutes

#This way the Patient ID is now the index variable and sorted    
main = main.set_index(['ID']).sort_index(ascending = True)

In [21]:
#Store main in another variable in case I messed up and didn't want to run that loop again
main_na = main

In [22]:
main.head()

Variable,Age,BUN,BaseExcess,Calcium,Chloride,Creatinine,DBP,FiO2,Gender,Glucose,...,Fibrinogen_max,Fibrinogen_ad,Bilirubin_direct,EtCO2,Bilirubin_direct_min,EtCO2_min,Bilirubin_direct_max,EtCO2_max,Bilirubin_direct_ad,EtCO2_ad
ID,,,,,,,,,,,,,,,,,,,,,
1.0,50.73,15.000,NaN,NaN,NaN,0.50,60.685,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,66.67,12.035,0.99,NaN,107.465,0.70,59.135,0.40,1.0,122.205,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,79.74,9.090,2.79,8.40,107.340,0.60,54.120,0.52,0.0,122.160,...,267.01,-52.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,35.18,NaN,NaN,9.30,NaN,0.75,77.710,NaN,1.0,175.330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,60.74,13.900,NaN,7.37,NaN,0.71,70.920,0.50,1.0,122.180,...,NaN,NaN,NaN,29.89,NaN,25.1,NaN,45.94,NaN,-0.007558


158 columns: 38*4=152 + 3 age (no avg diff age) + 3 ID (4th id is set as index, so doesn't count)

In [23]:
#Sort the columns so we have median, max, min for each variable
main = main.reindex(sorted(main.columns), axis=1)

* Remove the extra variables created to describe the demographic measurements (3 each for 4, so drop 12 columns total)

In [24]:
#Get rid of those extra columns
main.drop(['ID_min','ID_max','Gender_max','Gender_min','Unit1_min','Unit1_max','Unit2_min','Unit2_max',
           'ID_ad','Gender_ad','Unit1_ad','Unit2_ad'
          ], axis = 1, inplace=True)

In [25]:
main.head()

Variable,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
ID,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,50.73,0.0,50.73,50.73,NaN,NaN,...,NaN,NaN,13.200,0.000,13.20,13.20,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,66.67,0.0,66.67,66.67,NaN,NaN,...,0.0,1.0,8.085,-0.065,8.15,8.02,7.380,0.070000,7.45,7.31
3.0,NaN,NaN,NaN,NaN,79.74,0.0,79.74,79.74,NaN,NaN,...,0.0,1.0,10.805,1.780,13.94,6.82,7.430,-0.006364,7.51,7.37
4.0,NaN,NaN,NaN,NaN,35.18,0.0,35.18,35.18,NaN,NaN,...,1.0,0.0,15.000,0.000,15.00,15.00,NaN,NaN,NaN,NaN
5.0,36.0,0.0,36.0,36.0,60.74,0.0,60.74,60.74,58.0,0.0,...,NaN,NaN,9.770,-0.840,12.45,7.41,7.305,-0.007857,7.51,7.25


In [26]:
main.describe()

Variable,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
count,6753.000000,6753.000000,6753.000000,6753.000000,21634.000000,21634.0,21634.000000,21634.000000,6653.000000,6653.000000,...,13210.000000,13210.000000,19469.000000,19469.000000,19469.000000,19469.000000,9623.000000,9623.000000,9623.000000,9623.000000
mean,150.534849,-5.536598,203.034927,112.809129,61.612462,0.0,61.612462,61.612462,97.818131,-0.119482,...,0.506359,0.493641,11.030008,-0.055266,12.126674,10.070161,7.385613,0.003187,7.421957,7.344268
std,567.875008,136.258850,812.736250,411.709138,16.475634,0.0,16.475634,16.475634,108.595266,9.479324,...,0.499978,0.499978,5.970541,1.236460,7.204289,5.444595,0.065629,0.020605,0.069666,0.084850
min,4.000000,-3578.695000,4.000000,-286.850000,13.770000,0.0,13.770000,13.770000,6.955000,-286.495000,...,0.000000,0.000000,0.095000,-41.705000,0.100000,0.000000,6.770000,-0.305000,6.840000,6.710000
25%,21.000000,0.000000,21.100000,20.000000,51.050000,0.0,51.050000,51.050000,53.920000,0.000000,...,0.000000,0.000000,7.575000,-0.245000,8.100000,6.870000,7.350000,-0.001250,7.380000,7.300000
50%,35.000000,0.000000,36.000000,32.830000,63.285000,0.0,63.285000,63.285000,72.000000,0.000000,...,1.000000,0.000000,10.165000,0.000000,11.040000,9.300000,7.390000,0.000000,7.420000,7.350000
75%,73.000000,0.000000,80.000000,65.080000,74.010000,0.0,74.010000,74.010000,104.000000,0.000000,...,1.000000,1.000000,13.300000,0.126667,14.620000,12.200000,7.425000,0.010000,7.460000,7.400000
max,9869.010000,2239.200000,9869.010000,9869.010000,100.990000,0.0,100.990000,100.990000,3724.130000,288.020000,...,1.000000,1.000000,152.900000,19.405000,319.140000,152.900000,7.730000,0.210000,7.900000,7.730000


### Data Inputation: 

* Determining how we should best deal with missing values

In [27]:
main.shape
#21634 rows, 111 columns

(21634, 146)

In [28]:
#We need to get rid of the variables we dont have a lot of information on
main.isna().sum()

Variable
AST        14881
AST_ad     14881
AST_max    14881
AST_min    14881
Age            0
           ...  
WBC_min     2165
pH         12011
pH_ad      12011
pH_max     12011
pH_min     12011
Length: 146, dtype: int64

* In many of the papers found on the project, as well as emails with the GSI suggested that it may not make sense to use certain variables that don't have a lot of measurements. For this cutoff, I used 20%. SO, I removed a variable less than 20% of patients had this measure

In [29]:
#If we have 20% or less of a variable, get rid of that measurement; Try this initially
pt = round(21634 * .2)
21634 - pt
#if more than this NA, get rid of

17307

In [30]:
#Extracting Columns that have too many missing values
main.iloc[:,:60].isna().sum() > 17307

Variable
AST                     False
AST_ad                  False
AST_max                 False
AST_min                 False
Age                     False
Age_ad                  False
Age_max                 False
Age_min                 False
Alkalinephos            False
Alkalinephos_ad         False
Alkalinephos_max        False
Alkalinephos_min        False
BUN                     False
BUN_ad                  False
BUN_max                 False
BUN_min                 False
BaseExcess              False
BaseExcess_ad           False
BaseExcess_max          False
BaseExcess_min          False
Bilirubin_direct         True
Bilirubin_direct_ad      True
Bilirubin_direct_max     True
Bilirubin_direct_min     True
Bilirubin_total         False
Bilirubin_total_ad      False
Bilirubin_total_max     False
Bilirubin_total_min     False
Calcium                 False
Calcium_ad              False
Calcium_max             False
Calcium_min             False
Chloride                False
C

In [31]:
#Extracting Columns that have too many missing values
main.iloc[:,60:120].isna().sum() > 17307

Variable
Glucose_min      False
HCO3             False
HCO3_ad          False
HCO3_max         False
HCO3_min         False
HR               False
HR_ad            False
HR_max           False
HR_min           False
Hct              False
Hct_ad           False
Hct_max          False
Hct_min          False
Hgb              False
Hgb_ad           False
Hgb_max          False
Hgb_min          False
Hour             False
Hour_max         False
Hour_min         False
Lactate          False
Lactate_ad       False
Lactate_max      False
Lactate_min      False
MAP              False
MAP_ad           False
MAP_max          False
MAP_min          False
Magnesium        False
Magnesium_ad     False
Magnesium_max    False
Magnesium_min    False
O2Sat            False
O2Sat_ad         False
O2Sat_max        False
O2Sat_min        False
PTT              False
PTT_ad           False
PTT_max          False
PTT_min          False
PaCO2            False
PaCO2_ad         False
PaCO2_max        False
Pa

In [32]:
main.iloc[:,120:].isna().sum() > 17307

Variable
SBP              False
SBP_ad           False
SBP_max          False
SBP_min          False
SaO2             False
SaO2_ad          False
SaO2_max         False
SaO2_min         False
Temp             False
Temp_ad          False
Temp_max         False
Temp_min         False
TroponinI         True
TroponinI_ad      True
TroponinI_max     True
TroponinI_min     True
Unit1            False
Unit2            False
WBC              False
WBC_ad           False
WBC_max          False
WBC_min          False
pH               False
pH_ad            False
pH_max           False
pH_min           False
dtype: bool

* Therefore, we are going to remove these:
    + TroponinI
    + Fibrinogen
    + EtCO2
    + Bilirubin_direct 

In [33]:
#Remove the corresponding columns (have to also do mins and maxes)
main_edit = main.drop([
    'TroponinI','TroponinI_min','TroponinI_max', 'TroponinI_ad',
    'Fibrinogen', 'Fibrinogen_max','Fibrinogen_min', 'Fibrinogen_ad',
    'EtCO2', 'EtCO2_min','EtCO2_max', 'EtCO2_ad',
    'Bilirubin_direct_min', 'Bilirubin_direct_max', 'Bilirubin_direct', 'Bilirubin_direct_ad'
], axis=1)
main_edit.shape

(21634, 130)

* main_edit is now the new dataframe that we are working with, has the columns dropped. 

* We were given 38 variables initially. We then got 3 measured of each (114 total now), and added median of hour, max_hour, and min_hour (117 total). After dropping min/max of the 4 demographic variables (112 total now), removing our 4 undermeasured measurements, and setting the ID variable as index, we have 100 columns.

In [34]:
main_edit.head()

Variable,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
ID,,,,,,,,,,,,,,,,,,,,,
1.0,NaN,NaN,NaN,NaN,50.73,0.0,50.73,50.73,NaN,NaN,...,NaN,NaN,13.200,0.000,13.20,13.20,NaN,NaN,NaN,NaN
2.0,NaN,NaN,NaN,NaN,66.67,0.0,66.67,66.67,NaN,NaN,...,0.0,1.0,8.085,-0.065,8.15,8.02,7.380,0.070000,7.45,7.31
3.0,NaN,NaN,NaN,NaN,79.74,0.0,79.74,79.74,NaN,NaN,...,0.0,1.0,10.805,1.780,13.94,6.82,7.430,-0.006364,7.51,7.37
4.0,NaN,NaN,NaN,NaN,35.18,0.0,35.18,35.18,NaN,NaN,...,1.0,0.0,15.000,0.000,15.00,15.00,NaN,NaN,NaN,NaN
5.0,36.0,0.0,36.0,36.0,60.74,0.0,60.74,60.74,58.0,0.0,...,NaN,NaN,9.770,-0.840,12.45,7.41,7.305,-0.007857,7.51,7.25


In [35]:
main_edit.mean()

Variable
AST        150.534849
AST_ad      -5.536598
AST_max    203.034927
AST_min    112.809129
Age         61.612462
              ...    
WBC_min     10.070161
pH           7.385613
pH_ad        0.003187
pH_max       7.421957
pH_min       7.344268
Length: 130, dtype: float64

In [36]:
#main_2.head(2)

NameError: name 'main_2' is not defined

In [37]:
main_edit.median()

Variable
AST        35.000
AST_ad      0.000
AST_max    36.000
AST_min    32.830
Age        63.285
            ...  
WBC_min     9.300
pH          7.390
pH_ad       0.000
pH_max      7.420
pH_min      7.350
Length: 130, dtype: float64

* I first tried MICE inputation, but after evaluating readings/discussions, realized that too much missing data would impact this. Therefore, I deciced instead to do both a mean and median fill to deal with NAs, and see which one had the best impact

In [38]:
#Filling with median
main_1 = main_edit.fillna(main_edit.median())
#Filling with mean
main_2 = main_edit.fillna(main_edit.mean())

In [39]:
main_1.head(2)

Variable,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
ID,,,,,,,,,,,,,,,,,,,,,
1.0,35.0,0.0,36.0,32.83,50.73,0.0,50.73,50.73,72.0,0.0,...,1.0,0.0,13.200,0.000,13.20,13.20,7.39,0.00,7.42,7.35
2.0,35.0,0.0,36.0,32.83,66.67,0.0,66.67,66.67,72.0,0.0,...,0.0,1.0,8.085,-0.065,8.15,8.02,7.38,0.07,7.45,7.31


Now, we are going to input mean

In [40]:
main_1.shape

(21634, 130)

In [41]:
main_1.isna().sum()

Variable
AST        0
AST_ad     0
AST_max    0
AST_min    0
Age        0
          ..
WBC_min    0
pH         0
pH_ad      0
pH_max     0
pH_min     0
Length: 130, dtype: int64

In [42]:
main_2.shape

(21634, 130)

In [43]:
main_2.isna().sum()

Variable
AST        0
AST_ad     0
AST_max    0
AST_min    0
Age        0
          ..
WBC_min    0
pH         0
pH_ad      0
pH_max     0
pH_min     0
Length: 130, dtype: int64

# Modeling 

* Start with XGBoost, as this has been suggested as the best model

### XGBoost

* First, we must prepare the data for each of our different methods

In [44]:
main_1_train = pd.merge(main_1, train, on = 'ID').set_index('ID')
main_1_train.head()

,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min,Outcome
ID,,,,,,,,,,,,,,,,,,,,,
2.0,35.00,0.00,36.00,32.83,66.67,0.0,66.67,66.67,72.00,0.0,...,1.0,8.085,-0.065000,8.15,8.02,7.380,0.070000,7.45,7.31,0
3.0,35.00,0.00,36.00,32.83,79.74,0.0,79.74,79.74,72.00,0.0,...,1.0,10.805,1.780000,13.94,6.82,7.430,-0.006364,7.51,7.37,0
4.0,35.00,0.00,36.00,32.83,35.18,0.0,35.18,35.18,72.00,0.0,...,0.0,15.000,0.000000,15.00,15.00,7.390,0.000000,7.42,7.35,0
5.0,36.00,0.00,36.00,36.00,60.74,0.0,60.74,60.74,58.00,0.0,...,0.0,9.770,-0.840000,12.45,7.41,7.305,-0.007857,7.51,7.25,1
6.0,24.95,-1.04,25.99,23.91,77.32,0.0,77.32,77.32,70.18,0.0,...,0.0,16.340,-1.613333,17.44,12.60,7.390,0.000000,7.42,7.35,0


In [45]:
main_1_train.shape

(15144, 131)

In [46]:
main_1_test = pd.merge(main_1, test, on = 'ID').set_index('ID')
main_1_test.head()

,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min,Outcome,Score
ID,,,,,,,,,,,,,,,,,,,,,
1.0,35.000,0.000,36.00,32.83,50.73,0.0,50.73,50.73,72.000,0.000,...,13.2,0.000000,13.20,13.20,7.39,0.0,7.42,7.35,NaN,NaN
8.0,22.000,0.000,22.00,22.00,73.55,0.0,73.55,73.55,72.000,0.000,...,13.3,0.000000,13.30,13.30,7.39,0.0,7.42,7.35,NaN,NaN
9.0,35.000,0.000,36.00,32.83,52.71,0.0,52.71,52.71,72.000,0.000,...,10.6,0.000000,10.60,10.60,7.39,0.0,7.42,7.35,NaN,NaN
10.0,29.485,1.525,31.01,27.96,66.65,0.0,66.65,66.65,46.005,0.895,...,8.3,0.070000,9.96,8.09,7.39,0.0,7.42,7.35,NaN,NaN
11.0,33.000,0.000,33.00,33.00,84.08,0.0,84.08,84.08,72.000,0.000,...,14.1,-2.403333,17.45,10.24,7.39,0.0,7.42,7.35,NaN,NaN


In [47]:
main_1_test.shape

(6490, 132)

In [48]:
main_1_X_train = main_1_train.drop(['Outcome'], axis = 1)
main_1_y_train = main_1_train['Outcome']

main_1_X_test = main_1_test.drop(['Outcome', 'Score'], axis = 1)
main_1_y_test = main_1_test['Outcome']

In [49]:
main_1_X_train.shape

(15144, 130)

In [50]:
main_1_X_test.shape

(6490, 130)

In [51]:
main_1_y_train

ID
2.0        0
3.0        0
4.0        0
5.0        1
6.0        0
          ..
21628.0    0
21629.0    0
21630.0    0
21632.0    1
21633.0    1
Name: Outcome, Length: 15144, dtype: int64

In [52]:
main_1_y_test

ID
1.0       NaN
8.0       NaN
9.0       NaN
10.0      NaN
11.0      NaN
           ..
21617.0   NaN
21623.0   NaN
21627.0   NaN
21631.0   NaN
21634.0   NaN
Name: Outcome, Length: 6490, dtype: float64

## Now, we can use our model to evaluate using CV

##### First, using median fill

In [53]:
from sklearn.model_selection import KFold
import xgboost as xgb
from sklearn.model_selection import cross_val_score

In [54]:
#Initiate the classifier
model = xgb.XGBClassifier()

In [55]:
#K=10
kfold = KFold(n_splits=10, random_state =100, shuffle = True)

In [56]:
#Results from CV
results_a = cross_val_score(model, main_1_X_train, main_1_y_train, cv=kfold, scoring = 'roc_auc')

In [57]:
results_a.mean()

0.919312963146089

In [58]:
results_b = cross_val_score(model, main_1_X_train, main_1_y_train, cv=kfold, scoring = 'balanced_accuracy')

In [59]:
bac_1 = results_b.mean()
bac_1

0.7916910797294264

In [60]:
1-bac_1

0.20830892027057357

In [61]:
#Fitting our model with our training data
model.fit(main_1_X_train, main_1_y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [62]:
#Predictions
pred_main_1 = model.predict(main_1_X_test)
sum(pred_main_1)

598

In [74]:
pred_main_1

array([0, 0, 0, ..., 0, 0, 0])

In [63]:
#Scores
score_main_1 = model.predict_proba(main_1_X_test)
#6490 by 2 array for each test point

In [64]:
score_main_1.shape

(6490, 2)

In [71]:
prob_sep_main_1 = score_main_1[:,1]

In [73]:
#Probabilities of having sepsis
prob_sep_main_1

array([0.04671273, 0.04374389, 0.00122502, ..., 0.01446101, 0.00419097,
       0.00977409], dtype=float32)

####  What if we use mean fill

In [75]:
main_2_train = pd.merge(main_2, train, on = 'ID').set_index('ID')
main_2_test = pd.merge(main_2, test, on = 'ID').set_index('ID')

In [76]:
main_2_X_train = main_2_train.drop(['Outcome'], axis = 1)
main_2_y_train = main_2_train['Outcome']

main_2_X_test = main_2_test.drop(['Outcome', 'Score'], axis = 1)
main_2_y_test = main_2_test['Outcome']

In [77]:
main_2_X_train.head(2)

,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
ID,,,,,,,,,,,,,,,,,,,,,
2.0,150.534849,-5.536598,203.034927,112.809129,66.67,0.0,66.67,66.67,97.818131,-0.119482,...,0.0,1.0,8.085,-0.065,8.15,8.02,7.38,0.070000,7.45,7.31
3.0,150.534849,-5.536598,203.034927,112.809129,79.74,0.0,79.74,79.74,97.818131,-0.119482,...,0.0,1.0,10.805,1.780,13.94,6.82,7.43,-0.006364,7.51,7.37


In [78]:
main_2_X_test.head(2)

,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
ID,,,,,,,,,,,,,,,,,,,,,
1.0,150.534849,-5.536598,203.034927,112.809129,50.73,0.0,50.73,50.73,97.818131,-0.119482,...,0.506359,0.493641,13.2,0.0,13.2,13.2,7.385613,0.003187,7.421957,7.344268
8.0,22.000000,0.000000,22.000000,22.000000,73.55,0.0,73.55,73.55,97.818131,-0.119482,...,0.506359,0.493641,13.3,0.0,13.3,13.3,7.385613,0.003187,7.421957,7.344268


In [79]:
main_2_y_train.head(2)

ID
2.0    0
3.0    0
Name: Outcome, dtype: int64

In [80]:
model_2 = xgb.XGBClassifier()
kfold_2 = KFold(n_splits=10, random_state =100, shuffle = True)
results_2_a = cross_val_score(model_2, main_2_X_train, main_2_y_train, cv=kfold, scoring = 'roc_auc')

In [81]:
results_2_a.mean()

0.9211374560619964

In [82]:
results_2_b = cross_val_score(model_2, main_2_X_train, main_2_y_train, cv=kfold, scoring = 'balanced_accuracy')

In [83]:
bal_acc_2 = results_2_b.mean()

In [84]:
1-bal_acc_2

0.20559519950087102

In [85]:
#Fitting our model with our training data
model_2.fit(main_2_X_train, main_2_y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [86]:
#Predictions
pred_main_2 = model.predict(main_2_X_test)
sum(pred_main_2)

860

In [89]:
pred_main_2

array([0, 0, 0, ..., 0, 0, 0])

In [87]:
score_main_2 = model.predict_proba(main_2_X_test)

In [88]:
prob_sep_main_2 = score_main_2[:,1]

In [90]:
prob_sep_main_2

array([0.13805771, 0.17547996, 0.01948687, ..., 0.00902871, 0.0342835 ,
       0.01390529], dtype=float32)

* Therefore, although they are very similar, we get slightly better accuracy with main_2, which is the mean fill

#### Exporting Predictions

In [97]:
main_2_X_test

,AST,AST_ad,AST_max,AST_min,Age,Age_ad,Age_max,Age_min,Alkalinephos,Alkalinephos_ad,...,Unit1,Unit2,WBC,WBC_ad,WBC_max,WBC_min,pH,pH_ad,pH_max,pH_min
ID,,,,,,,,,,,,,,,,,,,,,
1.0,150.534849,-5.536598,203.034927,112.809129,50.73,0.0,50.73,50.73,97.818131,-0.119482,...,0.506359,0.493641,13.200000,0.000000,13.200000,13.200000,7.385613,0.003187,7.421957,7.344268
8.0,22.000000,0.000000,22.000000,22.000000,73.55,0.0,73.55,73.55,97.818131,-0.119482,...,0.506359,0.493641,13.300000,0.000000,13.300000,13.300000,7.385613,0.003187,7.421957,7.344268
9.0,150.534849,-5.536598,203.034927,112.809129,52.71,0.0,52.71,52.71,97.818131,-0.119482,...,0.000000,1.000000,10.600000,0.000000,10.600000,10.600000,7.385613,0.003187,7.421957,7.344268
10.0,29.485000,1.525000,31.010000,27.960000,66.65,0.0,66.65,66.65,46.005000,0.895000,...,0.000000,1.000000,8.300000,0.070000,9.960000,8.090000,7.385613,0.003187,7.421957,7.344268
11.0,33.000000,0.000000,33.000000,33.000000,84.08,0.0,84.08,84.08,97.818131,-0.119482,...,0.506359,0.493641,14.100000,-2.403333,17.450000,10.240000,7.385613,0.003187,7.421957,7.344268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21617.0,150.534849,-5.536598,203.034927,112.809129,44.84,0.0,44.84,44.84,97.818131,-0.119482,...,0.506359,0.493641,11.800000,0.000000,11.800000,11.800000,7.385613,0.003187,7.421957,7.344268
21623.0,1280.390000,-83.971429,1876.000000,187.500000,61.04,0.0,61.04,61.04,266.320000,-14.315714,...,1.000000,0.000000,17.990000,-0.605000,21.400000,16.080000,7.480000,0.006000,7.510000,7.420000
21627.0,150.534849,-5.536598,203.034927,112.809129,44.47,0.0,44.47,44.47,97.818131,-0.119482,...,1.000000,0.000000,11.030008,-0.055266,12.126674,10.070161,7.385613,0.003187,7.421957,7.344268


In [95]:
main_2_y_test

ID
1.0       NaN
8.0       NaN
9.0       NaN
10.0      NaN
11.0      NaN
           ..
21617.0   NaN
21623.0   NaN
21627.0   NaN
21631.0   NaN
21634.0   NaN
Name: Outcome, Length: 6490, dtype: float64

* We can see that ID 1 is in index 0, ID 8 is in index 1,.., and ID 21634 is in Index 6489.

* Our labels are in pred_main_2, and scores are in prob_sep_main_2

In [92]:
test

,ID,Outcome,Score
0,1,NaN,NaN
1,8,NaN,NaN
2,9,NaN,NaN
3,10,NaN,NaN
4,11,NaN,NaN
...,...,...,...
6485,21617,NaN,NaN
6486,21623,NaN,NaN
6487,21627,NaN,NaN
6488,21631,NaN,NaN


In [100]:
sum(pred_main_2 == 0)

5630

In [101]:
sum(pred_main_2 == 1)

860

In [104]:
max(prob_sep_main_2)

0.99998224

In [103]:
min(prob_sep_main_2)

0.00016845186

In [106]:
#Write to columns

test['Model_Outcome'] = pred_main_2
test['Model_Score'] = prob_sep_main_2

In [108]:
test.drop(['Outcome','Score'], axis = 1, inplace = True)

In [109]:
test

,ID,Model_Outcome,Model_Score
0,1,0,0.138058
1,8,0,0.175480
2,9,0,0.019487
3,10,0,0.006522
4,11,0,0.001731
...,...,...,...
6485,21617,0,0.038099
6486,21623,0,0.008935
6487,21627,0,0.009029
6488,21631,0,0.034283


In [113]:
#Testing that this matches
pred_main_2[151]

1

In [114]:
prob_sep_main_2[151]

0.99899596

In [117]:
test.iloc[151]

ID               428.000000
Model_Outcome      1.000000
Model_Score        0.998996
Name: 151, dtype: float64

In [123]:
#Writing Test to CSV

results = test.to_csv('test.csv', index = False)

In [124]:
pd.read_csv('test.csv')

,ID,Model_Outcome,Model_Score
0,1,0,0.138058
1,8,0,0.175480
2,9,0,0.019487
3,10,0,0.006522
4,11,0,0.001731
...,...,...,...
6485,21617,0,0.038099
6486,21623,0,0.008935
6487,21627,0,0.009029
6488,21631,0,0.034283


# Other Methods

These methods were also tested, but did not perform as well as the other models

### DecisionTree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree

In [ ]:
tree1 = DecisionTreeClassifier(random_state=0)

In [ ]:
kfold_3 = KFold(n_splits=10, random_state =100, shuffle = True)
results_3 = cross_val_score(tree1, main_1_X_train, main_1_y_train, cv=kfold, scoring = 'roc_auc')

In [ ]:
results_3.mean()

* What if we use main_2

In [ ]:
tree2 = DecisionTreeClassifier(random_state=0)
kfold_4 = KFold(n_splits=10, random_state =100, shuffle = True)
results_4 = cross_val_score(tree2, main_2_X_train, main_2_y_train, cv=kfold, scoring = 'roc_auc')

In [ ]:
results_4.mean()

* Therefore, we did not see an improvement

### RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=0)

In [ ]:
kfold_5 = KFold(n_splits=10, random_state =100, shuffle = True)
results_5 = cross_val_score(clf, main_1_X_train, main_1_y_train, cv=kfold, scoring = 'roc_auc')

In [ ]:
results_5.mean()

In [ ]:
clf2 = RandomForestClassifier(random_state=0)
kfold_6 = KFold(n_splits=10, random_state =100, shuffle = True)
results_6 = cross_val_score(clf, main_2_X_train, main_2_y_train, cv=kfold, scoring = 'roc_auc')

In [ ]:
results_6.mean()

* Therefore, we can conlude that our XGBoost model performed the best when using 10-Fold CV, and we will use the median inputation that was specified earlier.